In [1]:
import json
import emoji
import re
import nltk
import numpy as np
import tweets_processor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [17]:
# igonre this, this was the first file without regions
# parsing json file containing tweets
with open('tweets.json') as f_tweets:
    json_tweets = json.load(f_tweets)
#print(json_tweets[1]['text'])

tweets_text = [] # should we change this to numpy
tweets_place = []
for tweet in json_tweets:
    if (tweet['user']['geo_enabled'] == True):
        tweets_text.append(tweet['text'])
        tweets_place.append(tweet['place']['full_name'])

#print(tweets_text[1])
#print(len(tweets_text))
#print(len(tweets_place))

In [2]:
tweets_text, tweets_place = tweets_processor.get_tweets_from_csv()

In [3]:
s = 'test the 🤔 🙈 emojis, in 😌 here 💕👭'
s = ''.join(c for c in s if c not in emoji.UNICODE_EMOJI)
print(s)

test the   emojis, in  here 


In [3]:
#vectorizer = TfidfVectorizer(preprocessor = preprocessor, tokenizer=tokenizer)
# using bigrams
vectorizer = TfidfVectorizer(preprocessor = tweets_processor.preprocessor, tokenizer=tweets_processor.tokenizer,ngram_range=(2,2))
tweets_vectorized = vectorizer.fit_transform(tweets_text)

# length of total vocabulary
#print(len(vectorizer.vocabulary_))
print(tweets_vectorized.shape)


# count vectorizer
#count_vectorizer = CountVectorizer(preprocessor = preprocessor, tokenizer=tokenizer)
#using bigrams
count_vectorizer = CountVectorizer(preprocessor = tweets_processor.preprocessor, tokenizer=tweets_processor.tokenizer,ngram_range=(2,2))
tweets_count_vectorized = count_vectorizer.fit_transform(tweets_text)

# length of total vocabulary
#print(count_vectorizer.vocabulary_)
#print(count_vectorizer.get_feature_names())
#print(tweets_vectorized.shape)

(10000, 64881)


In [34]:
# splitting the data into train, dev, test
#train_data, train_labels = tweets_text[:700], tweets_place[:700] # ????see if we automatically split it proportinately irrespective of size 
#dev_data, dev_labels = tweets_text[700:901], tweets_place[700:901]
#test_data, test_labels = tweets_text[901:], tweets_place[901:]
print(tweets_text[1], tweets_vectorized[1], vectorizer.get_feature_names()[31054], vectorizer.get_feature_names()[1452], vectorizer.get_feature_names()[2172], tweets_place[1])

# using train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(tweets_vectorized, tweets_place, test_size=0.33, random_state=0)

#using count vectorized tweets
#train_data, test_data, train_labels, test_labels = train_test_split(tweets_count_vectorized, tweets_place, test_size=0.33, random_state=0)

When you like something on twit a lil pumpkin pops up🎃🙈   (0, 31054)	0.4548641685301028
  (0, 49677)	0.4548641685301028
  (0, 57029)	0.4548641685301028
  (0, 31209)	0.4354858003208915
  (0, 42986)	0.4354858003208915 america phonies amendment requires another cap cincinnati


In [36]:
train_data[0]

<1x62407 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [31]:
# training the model using logisticRegression with multinomial
####???? if there is a better library to use and a better classifier to use, should i svm, ridge classifier?

# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
lg.fit(train_data, train_labels)

print("Accuracy on test set: {:.02%}".format(lg.score(test_data, test_labels)))
# print(lg.score(test_data, test_labels))

#Ridge Classifier
rc = RidgeClassifier()
rc.fit(train_data, train_labels)

pred = rc.predict(test_data)

print("Accuracy on test set: {:.02%}".format(rc.score(test_data, test_labels)))
#print(rc.score(test_data, test_labels))
#print(classification_report(test_labels, pred))


Accuracy on test set: 15.21%
Accuracy on test set: 14.66%


In [32]:
# using naive bayes
nb = MultinomialNB()
nb.fit(train_data, train_labels)
y_pred = nb.predict(test_data)
acc = accuracy_score(test_labels, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

Accuracy on test set: 13.90%


In [27]:
# using MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(100, ),)
mlp.fit(train_data, train_labels)
y_pred = mlp.predict(test_data)
acc = accuracy_score(test_labels, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

Accuracy on test set: 13.39%


/Users/divyagorantla/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


<h4>Logic for the baseline model to get words for each region

In [ ]:
print(tweets_text[0], tweets_vectorized[1], vectorizer.get_feature_names()[1485], vectorizer.get_feature_names()[1452], vectorizer.get_feature_names()[2172], tweets_place[0])
vocab = vectorizer.get_feature_names()
place_word_score = defaultdict(lambda: defaultdict(lambda: 0.0))
word_place_score = defaultdict(lambda: defaultdict(lambda: 0.0))
word_score = dict()
for index, tweet in enumerate(tweets_vectorized):
    place = tweets_place[index]
    for word_index, score in zip(tweet.indices, tweet.data):
        word = vocab[word_index]
        existing_place = place_word_score.get(place, None)
        if existing_place is not None and existing_place.get(word, None) is not None:
            score = existing_place.get(word) + score
        # stores for each place what are all the words
        place_word_score[place][word] = score
        word_score_in_dict = word_score.get(word, None)
        # update the dict with the place and word only when there is no word in dict(word_score) with a score or if the score is greater for this place than the previous place
        # also update the word_score with new score
        if word_score_in_dict is None or word_score_in_dict < score:
            word_place_score[word] = {place:score}
            word_score[word] = score
print(word_place_score)